# Homework

# Respuestas:
1. Program the solution of unconstraint minimization for $T=80$. How many variables we need to solve the problem?

In [1]:
#import Pkg; Pkg.add("Optim")
using Optim, Parameters

Para realizar la tarea, se utilizarán los valores y el modelo dado en la clase de consumo-ahorro.

In [2]:
# Función de parámetros para T=80
function params(; T::Int=80, 
    β=0.96,           # Factor de descuento
    R=1.04,           # Tasa de retorno de los activos R = (1 + r), con r = 0.04
    ā=150)            # activos iniciales
    y = fill(100, T)  # Generamos un vector de ingresos com T=80, donde se asume que y=100 para todos los períodos
    return (β=β, R=R, ā=ā, y=y)
end

params (generic function with 1 method)

In [3]:
ck = params()

(β = 0.96, R = 1.04, ā = 150, y = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100  …  100, 100, 100, 100, 100, 100, 100, 100, 100, 100])

In [14]:
#Función de utilidad, asumiendo utilidad CRRA
function u(c; γ = 2.0)
    return c^(1 - γ) / (1 - γ)
end

# Función objetivo
function objective(ck, x)
    @unpack β, R, ā, y = ck  
    T = length(y)
    a = [0.0; x]  # a[1] = 0 y a[2] = x[1], a[3] = x[2], ..., a[T] = x[T-1]
    c = Vector{Float64}(undef, T)
    
    # definimos el comsumo para T=1 y despues para el resto de los tiempos
    c[1] = R * ā + y[1] - a[2]
    for t in 2:T
        if t < T
            c[t] = R * a[t] + y[t] - a[t + 1]
        else
            c[t] = R * a[t] + y[t]
        end
    end
    
    # Se calcula la suma de las utilidades
    utility_sum = 0.0
    for t in 1:T
        utility_sum += β^(t - 1) * u(c[t])
    end
    
    # Ahora devolvemos el valor negativo porque Optim minimiza por defecto
    return -utility_sum
end

# Se establece un punto inicial para la optimización, donde
T = 80
initial_guess = fill(150.0, T-1)  # Suposición inicial para a2, a3, ..., a80
ck = params()

# Optimizando
result = optimize(x -> objective(ck, x), initial_guess, BFGS())

# Se generan los resultados en optimal_a y optimal_c
optimal_a = Optim.minimizer(result)
optimal_c = Vector{Float64}(undef, T)

@unpack β, R, ā, y = ck

# se calculan los consumos óptimos
optimal_c[1] = R * ā + y[1] - optimal_a[1]
for t in 2:T
    if t < T
        optimal_c[t] = R * optimal_a[t-1] + y[t] - optimal_a[t]
    else
        optimal_c[t] = R * optimal_a[t-1] + y[t]
    end
end

# Se guardan los resultados óptimos
println("Optimal a values: ", optimal_a)
println("Optimal c values: ", optimal_c)


Optimal a values: [147.90600179970411, 145.81415551444655, 143.72497707400169, 141.63968628871825, 139.55693486737266, 137.4766815050033, 135.39961412165223, 133.3255342811221, 131.25426264586216, 129.18638083202902, 127.12100930616083, 125.05911531836881, 123.00091660889639, 120.94583103400448, 118.89486034999395, 116.84652869609309, 114.80234801749594, 112.76181473674828, 110.72552537447653, 108.69158943651453, 106.6628541235572, 104.63605519981732, 102.61439235808015, 100.59733183210835, 98.5837067011035, 96.57529065473511, 94.57159546960897, 92.57173512385118, 90.57566249922058, 88.58525482800623, 86.59882037220709, 84.61836350080124, 82.64303339894462, 80.67330445341365, 78.70892823237823, 76.75086669837037, 74.79898187065011, 72.85181982321697, 70.91084889096142, 68.97592845374426, 67.04734422473486, 65.12411468094146, 63.209152144385456, 61.30284357732924, 59.401763183402586, 57.508092550488975, 55.62080166342765, 53.74132792678867, 51.87209544463187, 50.01233730316156, 48.16021

Para resolver el problema se necesitan los valores de las variables $a$ y $c$ para un $T=80$, sabiendo que el activo $a$ para el $T=1$ esta dado, y y que en el $T=81$ el activo $a81=0$, tenemos que necesitamos $79 activos para los periodos desde $T=2 hasta T=80$. Ahora para el consumo $c$, se necesitan un total de 80, uno por cada tiempo para poder encontrar la solución cuando realizamos minimización sin restricción. 

2. Program the solution of constraint minimization for $T=80$. How many equations, variables and constraint we need?

In [6]:
#import Pkg; Pkg.add("JuMP")
#import Pkg; Pkg.add("Ipopt")

   Resolving package versions...
  No Changes to `C:\Users\Ivanna Morales\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Ivanna Morales\.julia\environments\v1.10\Manifest.toml`


In [8]:
using JuMP, Ipopt

#Se define función u y su retorno
function u(c; γ=2.0)
    return c^(1 - γ) / (1 - γ)
end

@unpack β, R, ā, y = ck

# Modelo de optimización
model = Model(Ipopt.Optimizer)
#Se define que el consumo no debe ser negativo en cada uno de los periodos
@variable(model, c[1:80] >= 0)  
@variable(model, a[1:81])  # Activos en cada periodo (a[1] corresponde a ā)

# Función objetivo
@NLobjective(model, Max, sum(β^(t-1) * u(c[t]) for t in 1:80))

# Se definen las restricciones
@constraint(model, a[1] == ā)
for t in 1:80
    if t < 80
        @constraint(model, a[t+1] == R * a[t] + y[t] - c[t])
    else
        @constraint(model, 0 == R * a[t] + y[t] - c[t])  # Valor terminal de activos
    end
end

# Se resuelve el modelo
optimize!(model)

# Se obtiene el valor de las variables
import JuMP.value

# se guarda la solución
println("Solución óptima:")
for t in 1:80
    println("c[$t] = ", value(c[t]))
end


┌ Warning: Function u automatically registered with 1 arguments.
│ 
│ Calling the function with a different number of arguments will result in an
│ error.
│ 
│ While you can safely ignore this warning, we recommend that you manually
│ register the function as follows:
│ ```Julia
│ model = Model()
│ register(model, :u, 1, u; autodiff = true)
│ ```
└ @ MathOptInterface.Nonlinear C:\Users\Ivanna Morales\.julia\packages\MathOptInterface\aJZbq\src\Nonlinear\operators.jl:430


This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:      240
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       80

Total number of variables............................:      161
                     variables with only lower bounds:       80
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       81
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -2.4045826e+03 1.50e+02 3.99e-02  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Las variables utilizadas, el consumo va a tener un total de $80$ esto dado que $T=80$. en el caso de los activos $a$ se utilizan 80 variables de activos, contabilizando en activo inicial (el cual es dado para el periodo 1). 

Para las restricciones, se utiliza una primera para fijar el nivel inicial de activos, a partir de esa, se realizan para los periodos faltantes, es decir, hasta $T=80$.

Y con las ecuaciones, haay 80 ecuaciones de activos. 

3. Program the solution of Euler equation system solving by $c_t$ for $T=80$. How many equations and variables we need?  

In [54]:
#import Pkg; Pkg.add("ForwardDiff")
using NLsolve, ForwardDiff    #se utiliza forwardDiff para poder obtener los valores reales del sistema de ecuaciones 

# Función de utilidad, asumiendo utilidad CRRA
function u(c; γ=2.0)
    return c > 0 ? c^(1 - γ) / (1 - γ) : -Inf  # Devuelve -Infinito para consumos no válidos, es decir, aquelos menores o iguales a 0
end

# Se deriva la función de utilidad respecto a c
function u_prime(c; γ=2.0)
    return c^(-γ)
end

# Se expresa el sistema de ecuaciones de Euler
function euler(ck, x)
    @unpack β, R, ā, y = ck
    
    T = length(y)
    c = x[1:T]
    a = zeros(T)
    
    a[1] = R * ā + y[1] - c[1]
    for t in 2:T
        a[t] = R * a[t-1] + y[t] - c[t]
    end
    
    eqs = similar(x)
    eqs[1] = u_prime(c[1]) - β * R * u_prime(c[2])
    for t in 2:T-1
        eqs[t] = u_prime(c[t]) - β * R * u_prime(c[t+1])
    end
    eqs[T] = a[T]  
    
    return eqs
end

# Resolemos el sistema de ecuaciones de Euler para T=80
ck = params(T=80)
initial_guess = fill(150.0, 80)  # Suposición inicial para c1, c2, ..., c80

result = nlsolve(x -> ForwardDiff.value.(euler(ck, x)), initial_guess)

# Se obtienen los valores óptimos del comsumo ct
optimal_c = result.zero

# Se guradan los resultados óptimos
println("Optimal c values: ", optimal_c)


Optimal c values: [108.09439966714906, 108.00789006594253, 107.92144884168547, 107.83507604574147, 107.74877172698612, 107.66253593158244, 107.57636870274874, 107.49027008051794, 107.40424010149492, 107.31827879860433, 107.23238620082854, 107.14656233295143, 107.06080721527559, 106.97512086334946, 106.88950328768549, 106.80395449345384, 106.71847448019865, 106.63306324151888, 106.54772076475318, 106.46244703066547, 106.37724201310441, 106.29210567867179, 106.2070379863766, 106.12203888727737, 106.03710832411622, 105.95224623096257, 105.86745253281782, 105.78272714523531, 105.69806997393043, 105.61348091436155, 105.52895985133323, 105.4445066585614, 105.36012119824672, 105.27580332063495, 105.19155286356809, 105.10736965202244, 105.02325349764389, 104.93920419826728, 104.85522153743062, 104.77130528387795, 104.68745519105029, 104.60367099657125, 104.5199524217182, 104.43629917088461, 104.3527109310348, 104.26918737114204, 104.18572814162276, 104.10233287375488, 104.01900117909396, 103.9

En este problema, se necesitan tantas ecuaciones de euler como la cantidad de tiempo que estemos estudiando, en este caso, se requieren de $80$ ecuaciones de euler para poder resolver el problema. Por el lado de las variables, se necesita el consumo para cada uno de los períodos, por lo tanto se requieren de $80$. Para los activos, tambien se requieren de $80$, sin embargo, el activo del periodo siguiente al ultimo (que se necesita para el sistema de ecuaciones de euler) debe ser $0$ por la condición terminal. 

4. Program the solution of Euler equation system solving by $a_t$ for $T=80$. How many equations and variables we need?

In [61]:
using NLsolve, Parameters

function params(; β = 0.96,
                R = 1.04,
                y = fill(100.0, 80),
                ā = 150.0)
    return (β=β, R=R, ā=ā, y=y)
end

# Se define la función de utilidad 
function u(c; γ=2.0)
    return c > 0 ? c^(1 - γ) / (1 - γ) : -Inf  # Devuelve -Infinito para consumos no válidos
end

# Se Deriva de la función de utilidad
function u_prime(c; γ=2.0)
    return c^(-γ)
end

# Sistema de ecuaciones de Euler para T=80
function euler(ck, x)
    @unpack β, R, ā, y = ck
    n = length(y)
    equations = []
    for t in 1:n-1
        push!(equations, u_prime(R * (t == 1 ? ā : x[t-1]) + y[t] - x[t]) - β * R * u_prime(R * x[t] + y[t+1] - x[t+1]))
    end
    push!(equations, u_prime(R * x[n-1] + y[n] - x[n]) - β * R * u_prime(R * x[n]))
    return equations
end

# Se resuelve el sistema de ecuaciones
ck = params()
initial_guess = fill(150.0, 80)  # Supongamos un valor inicial para a[t] = 150 para cada periodo
result = nlsolve(x -> euler(p, x), initial_guess, autodiff=:forward)  # Utimizamos la diferenciación automática

# Se obtienen los valores óptimos de consumo c[t]
consumos_optimos = []
for t in 1:length(result.zero)
    push!(consumos_optimos, p.R * (t == 1 ? p.ā : result.zero[t-1]) + p.y[t] - result.zero[t])
end

# Se guardan los resultados obtenidos
println("Consumos óptimos para T=80:")
for t in 1:length(consumos_optimos)
    println("c[$t] = ", consumos_optimos[t])
end


Consumos óptimos para T=80:
c[1] = 107.91466755437648
c[2] = 107.8283012599855
c[3] = 107.74200408628948
c[4] = 107.65577597796957
c[5] = 107.56961687975141
c[6] = 107.48352673640491
c[7] = 107.39750549274396
c[8] = 107.3115530936268
c[9] = 107.2256694839557
c[10] = 107.13985460867715
c[11] = 107.05410841278152
c[12] = 106.96843084130339
c[13] = 106.88282183932122
c[14] = 106.79728135195745
c[15] = 106.71180932437844
c[16] = 106.62640570179445
c[17] = 106.54107042945954
c[18] = 106.4558034526717
c[19] = 106.37060471677249
c[20] = 106.28547416714747
c[21] = 106.20041174922568
c[22] = 106.11541740848004
c[23] = 106.03049109042692
c[24] = 105.94563274062642
c[25] = 105.8608423046822
c[26] = 105.77611972824135
c[27] = 105.69146495699461
c[28] = 105.60687793667611
c[29] = 105.52235861306333
c[30] = 105.43790693197731
c[31] = 105.35352283928226
c[32] = 105.26920628088597
c[33] = 105.18495720273928
c[34] = 105.10077555083642
c[35] = 105.01666127121476
c[36] = 104.93261430995491
c[37] = 104.84

Se sigue manteniendo el $T=80$ de las preguntas anteriores, por lo que, igual a la pregunta 3, se van a tener 80 ecuaciones, una por cada período, y las variables va a tener la misma cantidad, una para cada período de estudio. 

5. Program the solution using bisection method (we review using Solow) for $T=80$. Explain the implementation. 

In [18]:
using Roots
using DataFrames
using Printf
using Plots

In [19]:
# Función de parámetros para T=80
function params(; T::Int=80, 
    β=0.96,           # Factor de descuento
    R=1.04,           # Tasa de retorno de los activos R = (1 + r), con r = 0.04
    ā=150)            # activos iniciales
    y = fill(100, T)  # Generamos un vector de ingresos con T=80
    return (β=β, R=R, ā=ā, y=y, T=T)
end

# Función para resolver el sistema de ecuaciones shooting para ct y at
function shooting_c(ck, c0, a0)
    @unpack β, R, ā, y, T = ck

    at = NaN * ones(T + 1)
    ct = NaN * ones(T + 1)
    at[1] = a0
    ct[1] = c0

    for t = 1:T
        y_deprec = y[t] + R * at[t]
        if ct[t] >= y_deprec
            break
        end
        ct[t+1] = ct[t] * (β * R)^(1 / 2)  # equation (1)
        at[t+1] = y[t] + R * at[t] - ct[t]  # equation (2)
    end
    ct, at
end



shooting_c (generic function with 1 method)

In [20]:
# Función de parámetros para T=80
function params(; T::Int=80, 
    β=0.96,           # Factor de descuento
    R=1.04,           # Tasa de retorno de los activos R = (1 + r), con r = 0.04
    ā=150)            # activos iniciales
    y = fill(100, T)  # Generamos un vector de ingresos con T=80
    return (β=β, R=R, ā=ā, y=y, T=T)
end

function bisection_c(ck, c0, a0; tol=1e-4, max_iter=500, a_ter=0, verbose=true)
    @unpack β, R, ā, y, T = ck

    # Limites iniciales y valor para c0
    c0_upper = y
    c0_lower = 0
    c0_vec = []

    i = 0
    while true
        ct, at = shooting_c(ck, c0, a0)

        # Error de calculo para el período terminal
        error = at[T] - a_ter

        # Se verifica si la condición terminal se satisface
        if abs(error) < tol
            if verbose
                println("Converged successfully on iteration ", i+1)
            end
            return ct, at, c0_vec
        end

        i += 1
        if i == max_iter
            if verbose
                println("Convergence failed.")
            end
            return ct, at, c0_vec
        end

        # Se actualizan los limites y el valor de c0 para las siguientes iteraciones 
        if error > 0
            c0_lower = c0
        else
            c0_upper = c0
        end

        c0 = (c0_lower + c0_upper) / 2
        push!(c0_vec, c0)
    end
end



bisection_c (generic function with 1 method)

Con el metodo de bisection estamos buscando un valor de c, que satisface en valor terminarl de los activos, es decir cuando $a=0$, es beneficioso (mientras este bien definido el metodo) por que permite llegar a la convergencia, es decir el nivel de consumo en el periodo final, y es preferida a shooting, ya que shooting lo que hace es disparar ciertos valores que se pueden acercar, más no da un valor preciso para poder resolver el modelo. 